In [2]:
import theano
import theano.tensor as T
import lasagne
import numpy as np
import matplotlib.pyplot as plt
import imageio
from tqdm import tqdm
import pickle
import pandas as pd
from glob import glob
import random
from time import time
from itertools import chain
import skimage.transform
import threading
from queue import Queue

%matplotlib inline

Using cuDNN version 5110 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:1E.0)


In [3]:
from lasagne.layers import InputLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import BatchNormLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import DenseLayer
from lasagne.nonlinearities import rectify, softmax


def build_simple_block(incoming_layer, names,
                       num_filters, filter_size, stride, pad,
                       use_bias=False, nonlin=rectify):
    """Creates stacked Lasagne layers ConvLayer -> BN -> (ReLu)
    Parameters:
    ----------
    incoming_layer : instance of Lasagne layer
        Parent layer
    names : list of string
        Names of the layers in block
    num_filters : int
        Number of filters in convolution layer
    filter_size : int
        Size of filters in convolution layer
    stride : int
        Stride of convolution layer
    pad : int
        Padding of convolution layer
    use_bias : bool
        Whether to use bias in conlovution layer
    nonlin : function
        Nonlinearity type of Nonlinearity layer
    Returns
    -------
    tuple: (net, last_layer_name)
        net : dict
            Dictionary with stacked layers
        last_layer_name : string
            Last layer name
    """
    names = list(names)
    net = []
    net.append((
            names[0],
            ConvLayer(incoming_layer, num_filters, filter_size, stride, pad,
                      flip_filters=False, nonlinearity=None) if use_bias
            else ConvLayer(incoming_layer, num_filters, filter_size, stride, pad, b=None,
                           flip_filters=False, nonlinearity=None)
        ))

    net.append((
            names[1],
            BatchNormLayer(net[-1][1])
        ))
    if nonlin is not None:
        net.append((
            names[2],
            NonlinearityLayer(net[-1][1], nonlinearity=nonlin)
        ))

    return dict(net), net[-1][0]


def build_residual_block(incoming_layer, ratio_n_filter=1.0, ratio_size=1.0, has_left_branch=False,
                         upscale_factor=4, ix=''):
    """Creates two-branch residual block
    Parameters:
    ----------
    incoming_layer : instance of Lasagne layer
        Parent layer
    ratio_n_filter : float
        Scale factor of filter bank at the input of residual block
    ratio_size : float
        Scale factor of filter size
    has_left_branch : bool
        if True, then left branch contains simple block
    upscale_factor : float
        Scale factor of filter bank at the output of residual block
    ix : int
        Id of residual block
    Returns
    -------
    tuple: (net, last_layer_name)
        net : dict
            Dictionary with stacked layers
        last_layer_name : string
            Last layer name
    """
    simple_block_name_pattern = ['res%s_branch%i%s', 'bn%s_branch%i%s', 'res%s_branch%i%s_relu']

    net = {}

    # right branch
    net_tmp, last_layer_name = build_simple_block(
        incoming_layer, map(lambda s: s % (ix, 2, 'a'), simple_block_name_pattern),
        int(lasagne.layers.get_output_shape(incoming_layer)[1]*ratio_n_filter), 1, int(1.0/ratio_size), 0)
    net.update(net_tmp)

    net_tmp, last_layer_name = build_simple_block(
        net[last_layer_name], map(lambda s: s % (ix, 2, 'b'), simple_block_name_pattern),
        lasagne.layers.get_output_shape(net[last_layer_name])[1], 3, 1, 1)
    net.update(net_tmp)

    net_tmp, last_layer_name = build_simple_block(
        net[last_layer_name], map(lambda s: s % (ix, 2, 'c'), simple_block_name_pattern),
        lasagne.layers.get_output_shape(net[last_layer_name])[1]*upscale_factor, 1, 1, 0,
        nonlin=None)
    net.update(net_tmp)

    right_tail = net[last_layer_name]
    left_tail = incoming_layer

    # left branch
    if has_left_branch:
        net_tmp, last_layer_name = build_simple_block(
            incoming_layer, map(lambda s: s % (ix, 1, ''), simple_block_name_pattern),
            int(lasagne.layers.get_output_shape(incoming_layer)[1]*4*ratio_n_filter), 1, int(1.0/ratio_size), 0,
            nonlin=None)
        net.update(net_tmp)
        left_tail = net[last_layer_name]

    net['res%s' % ix] = ElemwiseSumLayer([left_tail, right_tail], coeffs=1)
    net['res%s_relu' % ix] = NonlinearityLayer(net['res%s' % ix], nonlinearity=rectify)

    return net, 'res%s_relu' % ix


def build_model():
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    sub_net, parent_layer_name = build_simple_block(
        net['input'], ['conv1', 'bn_conv1', 'conv1_relu'],
        64, 7, 2, 3, use_bias=True)
    net.update(sub_net)
    net['pool1'] = PoolLayer(net[parent_layer_name], pool_size=3, stride=2, pad=0, mode='max', ignore_border=False)
    block_size = list('abc')
    parent_layer_name = 'pool1'
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1, 1, True, 4, ix='2%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='2%s' % c)
        net.update(sub_net)

    block_size = list('abcd')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='3%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='3%s' % c)
        net.update(sub_net)

    block_size = list('abcdef')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='4%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='4%s' % c)
        net.update(sub_net)

    block_size = list('abc')
    for c in block_size:
        if c == 'a':
            sub_net, parent_layer_name = build_residual_block(
                net[parent_layer_name], 1.0/2, 1.0/2, True, 4, ix='5%s' % c)
        else:
            sub_net, parent_layer_name = build_residual_block(net[parent_layer_name], 1.0/4, 1, False, 4, ix='5%s' % c)
        net.update(sub_net)
    net['pool5'] = PoolLayer(net[parent_layer_name], pool_size=7, stride=1, pad=0,
                             mode='average_exc_pad', ignore_border=False)
    net['fc1000'] = DenseLayer(net['pool5'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc1000'], nonlinearity=softmax)

    return net

In [4]:
model = build_model()

In [5]:
weights = pickle.load(open('resnet50.pkl', 'rb'), encoding='latin1')
lasagne.layers.set_all_param_values(model['prob'], weights['values'])

In [9]:
newmodel = {}
newmodel['dense1'] = DenseLayer(model['pool5'], 1000)
newmodel['dense2'] = DenseLayer(newmodel['dense1'], 1000)
newmodel['prob'] = DenseLayer(newmodel['dense2'], 101, nonlinearity=softmax)

In [38]:
weights = sum([x.get_params(trainable=True) for x in newmodel.values()], [])
weights.extend(sum([x.get_params(trainablweightse=True) for k, x in model.items() if k[:4] == 'res5'], []))
weights

[W, b, W, b, W, b, W, W, W, W, W, W, W, W, W, W]

In [39]:
netinput = T.tensor4()
netoutput = T.matrix()
nettarget = T.ivector()

pred = lasagne.layers.get_output(newmodel['prob'], netinput)
pred_det = lasagne.layers.get_output(newmodel['prob'], netinput, deterministic=True)

loss = lasagne.objectives.categorical_crossentropy(pred, nettarget).mean()
final_acc = lasagne.objectives.categorical_accuracy(pred_det, nettarget).mean()

updates = lasagne.updates.adam(loss, weights, learning_rate=0.001)

/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/ubuntu/env/src/lasagne/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


In [40]:
train = theano.function([netinput, nettarget], loss, updates=updates)
val = theano.function([netinput, nettarget], final_acc)
pred = theano.function([netinput], pred_det)

In [41]:
def data_reader(d, firstchar):
    for filename in glob(d + '/' + firstchar + '*.avi'):
        yield imageio.get_reader(filename), filename[len(d) + 1:]

def random_frames(it, cnt=10):
    for r, fname in it:
        try:
            yield [(r.get_data(random.randint(0, r.get_length() - 1)), fname) for _ in range(cnt)]
        except:
            pass

def remove_batches(it):
    for r in it:
        for q in r:
            yield q

def resizer(it):
    for r, c in it:
        im = skimage.transform.resize(r, (224, 224), mode='constant')
        im = im.swapaxes(2, 1).swapaxes(1, 0)
        yield im, c

def replace_filename_with_class(it, df):
    for r, fname in it:
        yield r, df[df.filename==fname].classnum.values[0]

def local_shuffler(it, size):
    s = []
    for i in it:
        s.append(i)
        if len(s) > size:
            yield s.pop(random.randint(0, len(s) - 1))
    while len(s) > size:
        yield s.pop(random.randint(0, len(s) - 1))
    

def batcher(it, size):
    batchx, batchy = [], []
    for x, y in it:
        batchx.append(x)
        batchy.append(y)
        if len(batchx) == size:
            yield np.array(batchx).astype(np.float32), batchy
            batchx, batchy = [], []
    if len(batchy):
        yield np.array(batchx).astype(np.float32), batchy

def threaded_generator(generator, num_cached=3):
    # this code is written by jan Schluter
    # copied from https://github.com/benanne/Lasagne/issues/12
    queue = Queue(maxsize=num_cached)
    sentinel = object()  # guaranteed unique reference

    # define producer (putting items into queue)
    def producer():
        for item in generator:
            queue.put(item)
        queue.put(sentinel)

    # start producer (in a background thread)
    thread = threading.Thread(target=producer)
    thread.daemon = True
    thread.start()

    # run as consumer (read items from queue, in current thread)
    item = queue.get()
    while item is not sentinel:
        yield item
        queue.task_done()
        item = queue.get()

In [17]:
train_df = pd.read_csv('train_gt.csv')

In [18]:
train_log, val_log = [], []

In [ ]:
for ep in range(1):
    t = time()
    
    err = 0.
    batches = 0
    generator = threaded_generator(batcher(local_shuffler(
        replace_filename_with_class(
        resizer(
        remove_batches(
        random_frames(
            data_reader('action-recognition-train', '[1-9]'),
            cnt=1))), train_df), size=100), size=100))
    for bx, by in tqdm(generator):
        err += train(bx, by)
        batches += 1
    train_log.append(err / batches)
    err = 0.
    batches = 0
    generator = threaded_generator(batcher(
        replace_filename_with_class(
        resizer(
        remove_batches(
        random_frames(
            data_reader('action-recognition-train', '0'),
            cnt=1))), train_df), size=200))
    for bx, by in tqdm(generator):
        err += val(bx, by)
        batches += 1
    val_log.append(err / batches)
    
    print('epoch: {}\ttook: {}\ttrain_loss: {}\tval_loss: {}'.format(ep, time() - t, train_log[-1], val_log[-1]))

18it [08:29, 26.83s/it]

In [ ]:
result = {}
cnt_means = 4
generator = batcher(
        resizer(
        remove_batches(
        random_frames(
            data_reader('action-recognition-test', ''),
            cnt=cnt_means))), size=200 * cnt_means)
for bx, by in tqdm(generator):
    p = pred(bx).reshape([-1, cnt_means, 101]).mean(axis=1).argmax(axis=1)
    for fname, res in zip(by[::cnt_means], p):
        result[fname] = res



0it [00:00, ?it/s]

1it [01:55, 115.85s/it]

2it [03:52, 116.15s/it]

3it [05:47, 115.72s/it]

5it [09:34, 114.67s/it]

6it [11:29, 114.97s/it]

7it [13:23, 114.66s/it]

8it [15:18, 114.50s/it]

9it [17:12, 114.56s/it]

10it [19:08, 115.05s/it]

11it [21:04, 115.26s/it]

12it [22:58, 114.81s/it]

13it [24:53, 114.99s/it]

14it [26:48, 114.88s/it]

15it [28:44, 115.16s/it]

16it [30:39, 115.19s/it]

17it [32:36, 115.86s/it]

18it [34:31, 115.34s/it]

19it [35:43, 102.50s/it]



In [47]:
with open('result.csv', 'w') as f:
    f.write('filename,classnum\n')
    for fname, cls in result.items():
        f.write(fname + ',' + str(cls) + '\n')

In [44]:
pickle.dump(lasagne.layers.get_all_param_values(newmodel['prob']), open('weights.pcl', 'wb'))

In [ ]:
lasagne.layers.set_all_param_values(newmodel['prob'], pickle.loads(open('weights.pcl', 'rb'))

In [46]:
1+1

2